In [718]:
import re

Step = tuple[str | None, str | None, str | None, str]
Workflow = list[Step]
Part = dict[str, int]


def parse_workflow(worfklow_string: str) -> tuple:
    steps = []
    for value in worfklow_string.split(","):
        matches = re.match(r"(?:([a-zA-Z]+)([\>\<])(\d+):)?([a-zA-Z]+)$", value)
        steps.append(matches.groups())

    return steps


def parse(input_string: str) -> tuple[dict[Workflow], list[Part]]:
    sections = input_string.split("\n\n")
    worflows = {}
    for line in sections[0].splitlines():
        matches = re.match(r"(\w+)\{(.+)\}", line)
        worflows[matches.group(1)] = parse_workflow(matches.group(2))

    parts = []
    for line in sections[1].splitlines():
        part = {}
        for value in re.finditer(r"(\w)=(\d+)", line):
            part[value.group(1)] = int(value.group(2))

        parts.append(part)

    return (worflows, parts)

In [719]:
import math

DEFAULT_RANGES = {
    "x": (1, 4000),
    "m": (1, 4000),
    "a": (1, 4000),
    "s": (1, 4000),
}


def handle_step(
    step: Step,
    ranges: dict[tuple[int, int]],
):
    id, operator, value, next_workflow = step
    if id is None:
        return (ranges, None, next_workflow)

    if operator == "<":
        pass_ranges = ranges.copy()
        pass_ranges[id] = (ranges[id][0], min(int(value) - 1, ranges[id][1]))
        fail_ranges = ranges.copy()
        fail_ranges[id] = (max(int(value), ranges[id][0]), ranges[id][1])
        return (pass_ranges, fail_ranges, next_workflow)

    elif operator == ">":
        pass_ranges = ranges.copy()
        pass_ranges[id] = (max(int(value) + 1, ranges[id][0]), ranges[id][1])
        fail_ranges = ranges.copy()
        fail_ranges[id] = (ranges[id][0], min(int(value), ranges[id][1]))
        return (pass_ranges, fail_ranges, next_workflow)

    raise Exception("Unknown operator")


def iterate(
    workflows: list[Workflow],
    current_workflow: str = "in",
    ranges: dict[tuple[int, int]] = DEFAULT_RANGES,
):
    if current_workflow == "A":
        return [ranges]

    if current_workflow == "R":
        return []

    steps = workflows[current_workflow]
    results = []
    for step in steps:
        next_ranges, fail_ranges, next_workflow = handle_step(step, ranges)
        results += iterate(
            workflows=workflows,
            current_workflow=next_workflow,
            ranges=next_ranges,
        )
        if fail_ranges:
            ranges = fail_ranges

    return results


def is_part_in_range(part: Part, range: tuple[tuple]) -> bool:
    for key, value in part.items():
        if value < range[key][0] or value > range[key][1]:
            return False

    return True


def get_matching_ranges(part: Part, range_list: list[tuple[tuple]]) -> bool:
    output = []
    for range in range_list:
        if is_part_in_range(part, range):
            output.append(range)

    return output


def get_part_value(part: Part, key: str) -> int:
    return sum(part.values())


def get_possible_values_count(range_list: list[dict[tuple[int, int]]]) -> int:
    return sum(
        math.prod([value[1] - value[0] + 1 for value in range.values()])
        for range in range_list
    )

In [720]:
test_input = """\
px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}"""

In [721]:
workflows, parts = parse(test_input)
range_list = iterate(workflows)

assert (
    sum(sum(part.values()) for part in parts if get_matching_ranges(part, range_list))
    == 19114
)

In [722]:
workflows, parts = parse(open("19.txt").read())
range_list = iterate(workflows)

value = sum(
    sum(part.values()) for part in parts if get_matching_ranges(part, range_list)
)


print(f"Part 1: {value}")
assert value == 352052

Part 1: 352052


In [723]:
workflows, parts = parse(test_input)
range_list = iterate(workflows)

print(get_possible_values_count(range_list))
assert get_possible_values_count(range_list) == 167409079868000

167409079868000


In [724]:
workflows, parts = parse(open("19.txt").read())
range_list = iterate(workflows)
value = get_possible_values_count(range_list)

print(f"Part 2: {value}")
assert value == 116606738659695

Part 2: 116606738659695
